In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('max_columns', 8, 'max_rows', 20)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,...,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,...,1905,65.0,0,0
1,1,1,4,UA,...,1333,-13.0,0,0
2,1,1,4,MQ,...,1453,35.0,0,0
3,1,1,4,AA,...,1935,-7.0,0,0
4,1,1,4,WN,...,2225,39.0,0,0


### Grouping/aggregate on single column

In [16]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [17]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [18]:
type(flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean'))

pandas.core.series.Series

In [19]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [20]:
grouped = flights.groupby('AIRLINE')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [27]:
try:
    flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.sqrt)
except ValueError as error:
    print('Only aggregate methods are allowed for agg')
    error

Only aggregate methods are allowed for agg


ValueError('Must produce aggregated value')

### grouping mutiple columns

In [31]:
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].sum().head(20)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
         6          21
         7          29
AS       1           0
         2           0
         3           0
         4           0
         5           0
         6           0
         7           0
B6       1           0
         2           1
         3           0
         4           0
         5           0
         6           0
Name: CANCELLED, dtype: int64

In [37]:
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head()

<ipython-input-37-a107c84a44e3>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head()


CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786

In [42]:
flights.groupby(['AIRLINE', 'WEEKDAY'])['SCHED_ARR', 'DIVERTED'].agg(['sum', 'min']).head(10)

<ipython-input-42-f310cd33ec08>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  flights.groupby(['AIRLINE', 'WEEKDAY'])['SCHED_ARR', 'DIVERTED'].agg(['sum', 'min']).head(10)


SCHED_ARR     DIVERTED    
                      sum min      sum min
AIRLINE WEEKDAY                           
AA      1         2018786   3        6   0
        2         1906451   4        2   0
        3         2082989   1        2   0
        4         2044815   3        5   0
        5         1965188   1        1   0
        6         1734700   6        9   0
        7         2015023   2        1   0
AS      1          185654   9        0   0
        2          155258   9        0   0
        3          180524  27        0   0

In [44]:
groupin_cols = ['ORG_AIR', 'DEST_AIR']
agg_dict = {'CANCELLED' : ['sum', 'mean', 'size'],
            'AIR_TIME' : ['mean', 'var']}

In [51]:
flights.groupby(groupin_cols).agg(agg_dict)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

In [53]:
flights.groupby(groupin_cols).sum() ## aggregation applied to remaining columns by default

MONTH   DAY  WEEKDAY  SCHED_DEP  ...  SCHED_ARR  ARR_DELAY  \
ORG_AIR DEST_AIR                                   ...                         
ATL     ABE         167   519      132      46567  ...      52916      185.0   
        ABQ          98   189       62      25426  ...      27960        4.0   
        ABY         136   345       87      31253  ...      32758      391.0   
        ACY          40    72       11       8972  ...      10096       64.0   
        AEX         275   627      160      59674  ...      62361      301.0   
...                 ...   ...      ...        ...  ...        ...        ...   
SFO     SNA         780  1863      419     172954  ...     191323      389.0   
        STL         144   307       72      24558  ...      36147      362.0   
        SUN          59   141       47      14429  ...      17203      139.0   
        TUS         117   254       79      30202  ...      35549      311.0   
        XNA          22    35        5       2660  ...       3834      -57.0   

                  DIVERTED  CANCELLED  
ORG_AIR DEST_AIR                       
ATL     ABE              0          0  
        ABQ              0          0  
        ABY              0          0  
        ACY              0          0  
        AEX              0          0  
...                    ...        ...  
SFO     SNA              0          4  
        STL              0          0  
        SUN              0          0  
        TUS              0          0  
        XNA              0          0  

[1130 rows x 11 columns]

### removing the multi index after groupby

In [55]:
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY']) \
                        .agg({'DIST': ['sum', 'mean'], 'ARR_DELAY': ['min', 'max']}) \
                        .astype(int)

In [56]:
airline_info.head(7)

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
        6        1265340  1124       -50  858
        7        1461906  1100       -49  626

In [57]:
airline_info.columns

MultiIndex([(     'DIST',  'sum'),
            (     'DIST', 'mean'),
            ('ARR_DELAY',  'min'),
            ('ARR_DELAY',  'max')],
           )

In [58]:
level0 = airline_info.columns.get_level_values(0)

In [59]:
level1 = airline_info.columns.get_level_values(1)

In [60]:
level0

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [61]:
level1

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [62]:
airline_info.columns = level0 + '_' + level1

In [63]:
airline_info.columns

Index(['DIST_sum', 'DIST_mean', 'ARR_DELAY_min', 'ARR_DELAY_max'], dtype='object')

In [64]:
airline_info.head(7)

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
        6         1265340       1124            -50            858
        7         1461906       1100            -49            626

In [65]:
airline_info.reset_index().head(7)

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
5,AA,6,1265340,1124,-50,858
6,AA,7,1461906,1100,-49,626


In [71]:
groupby_without_index = flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

In [72]:
groupby_without_index.index   # Index not created with column

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

In [73]:
groupby_without_index.head(7)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0


### Using customized aggregate functions

In [74]:
college = pd.read_csv('data/college.csv')

In [79]:
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(0).head(10)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
CA,3518.0,6709.0
CO,2325.0,4670.0
CT,1874.0,2871.0
DC,2645.0,3225.0


In [81]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [82]:
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1).head(10)

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
CA    6.1
CO    5.0
CT    5.6
DC    2.4
DE    3.5
Name: UGDS, dtype: float64

In [85]:
college.groupby('STABBR')[['UGDS', 'SATVRMID']] \
        .agg(max_deviation).round(1).head(5)

,UGDS,SATVRMID
STABBR,,
AK,2.6,NaN
AL,5.8,1.6
AR,6.3,2.2
AS,NaN,NaN
AZ,9.9,1.9


In [87]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID'] \
        .agg([max_deviation, 'mean', 'std']).round(1).head()

<ipython-input-87-aea8f06b4f6a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID'] \


UGDS                      SATVRMID  ...        \
                max_deviation    mean     std max_deviation  ...   std   
STABBR RELAFFIL                                              ...         
AK     0                  2.1  3508.9  4539.5           NaN  ...   NaN   
       1                  1.1   123.3   132.9           NaN  ...   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  ...  56.5   
       1                  2.4   979.7   870.8           1.5  ...  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  ...  37.9   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0  

[5 rows x 9 columns]

In [88]:
max_deviation.__name__

'max_deviation'

In [89]:
max_deviation.__name__ = 'Max deviation'

In [90]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID'] \
        .agg([max_deviation, 'mean', 'std']).round(1).head()

<ipython-input-90-aea8f06b4f6a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID'] \


UGDS                      SATVRMID  ...        \
                Max deviation    mean     std Max deviation  ...   std   
STABBR RELAFFIL                                              ...         
AK     0                  2.1  3508.9  4539.5           NaN  ...   NaN   
       1                  1.1   123.3   132.9           NaN  ...   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  ...  56.5   
       1                  2.4   979.7   870.8           1.5  ...  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  ...  37.9   

                     SATMTMID               
                Max deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0  

[5 rows x 9 columns]

### Passing multiple args to cusotmized aggregate method

In [93]:
groupby_obj = college.groupby(['STABBR', 'RELAFFIL'])

In [94]:
import inspect
inspect.signature(groupby_obj.agg)

<Signature (func=None, *args, engine=None, engine_kwargs=None, **kwargs)>

In [95]:
def pct_between_1_3k(s):
    return s.between(1000, 3000).mean()

In [99]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
        .agg(pct_between_1_3k).head(5)

STABBR  RELAFFIL
AK      0           0.142857
        1           0.000000
AL      0           0.236111
        1           0.333333
AR      0           0.279412
Name: UGDS, dtype: float64

In [100]:
def pct_between(s, low, high): # customized boundaries
    return s.between(low, high).mean()

In [102]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
        .agg(pct_between, 1000, 3000).head(5)

STABBR  RELAFFIL
AK      0           0.142857
        1           0.000000
AL      0           0.236111
        1           0.333333
AR      0           0.279412
Name: UGDS, dtype: float64

#### How to use multiple argument methods combined with other aggregate methods

In [108]:
try:
    college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
            .agg(['mean', pct_between], low=1000, high=3000)
except TypeError as error:
    error

TypeError("pct_between() missing 2 required positional arguments: 'low' and 'high'")

#### this problem can be solved by creating customizable functions ( decorators concept in python)

In [109]:
def make_agg_func(func, name, *args, **kwargs):
    def wrapper(x):
        return func(x, *args, **kwargs)
    wrapper.__name__ = name
    return wrapper

In [110]:
my_agg1 = make_agg_func(pct_between, 'pct_1_3k', low=1000, high=3000)
my_agg2 = make_agg_func(pct_between, 'pct_10_30k', 10000, 30000)

In [111]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
        .agg(['mean', my_agg1, my_agg2]).head()

mean  pct_1_3k  pct_10_30k
STABBR RELAFFIL                                   
AK     0         3508.857143  0.142857    0.142857
       1          123.333333  0.000000    0.000000
AL     0         3248.774648  0.236111    0.083333
       1          979.722222  0.333333    0.000000
AR     0         1793.691176  0.279412    0.014706